In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transformers

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the model with 15 fully connected layers
class FullyConnectedNN(nn.Module):
    def __init__(self):
        super(FullyConnectedNN, self).__init__()
        
        # Define 15 fully connected layers
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        # self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        # self.fc5 = nn.Linear(128, 128)
        # self.fc6 = nn.Linear(128, 64)
        # self.fc7 = nn.Linear(64, 64)
        # self.fc8 = nn.Linear(64, 32)
        # self.fc9 = nn.Linear(32, 32)
        # self.fc10 = nn.Linear(32, 16)
        # self.fc11 = nn.Linear(16, 16)
        # self.fc12 = nn.Linear(16, 8)
        # self.fc13 = nn.Linear(8, 8)
        # self.fc14 = nn.Linear(8, 4)
        # self.fc15 = nn.Linear(4, 4)
        
        # Output layer for 10 classes (digits 0-9)
        self.out = nn.Linear(128, 10)

    def forward(self, x):
        # Flatten the input (28x28) to a vector (784)
        x = x.view(-1, 784)
        
        # Apply different activation functions between each layer
        x = F.leaky_relu(self.fc1(x))        # ReLU activation for layer 1
        x = F.leaky_relu(self.fc2(x))  # Leaky ReLU for layer 2
        # x = F.relu(self.fc3(x)) # Sigmoid activation for layer 3
        x = F.leaky_relu(self.fc4(x))        # Tanh activation for layer 4
        # x = F.relu(self.fc5(x))        # ReLU activation for layer 5
        # x = F.leaky_relu(self.fc6(x))  # Leaky ReLU for layer 6
        # x = torch.sigmoid(self.fc7(x)) # Sigmoid activation for layer 7
        # x = F.tanh(self.fc8(x))        # Tanh activation for layer 8
        # x = F.relu(self.fc9(x))        # ReLU activation for layer 9
        # x = F.leaky_relu(self.fc10(x)) # Leaky ReLU for layer 10
        # x = torch.sigmoid(self.fc11(x))# Sigmoid activation for layer 11
        # x = F.tanh(self.fc12(x))       # Tanh activation for layer 12
        # x = F.relu(self.fc13(x))       # ReLU activation for layer 13
        # x = F.leaky_relu(self.fc14(x)) # Leaky ReLU for layer 14
        # x = torch.sigmoid(self.fc15(x))# Sigmoid activation for layer 15
        
        # Output layer (for classification, we'll apply log softmax)
        x = F.softmax(self.out(x), dim=1)
        
        return x

# Hyperparameters
batch_size = 5000
learning_rate = 0.001
epochs = 5

# Load MNIST data (train and test datasets)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function and optimizer
model = FullyConnectedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Calculate loss
        loss = criterion(output, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Print progress
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Test loop
def test(model, device, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():  # Disable gradient computation for testing
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Forward pass
            output = model(data)
            
            # Sum up batch loss
            test_loss += criterion(output, target).item()
            
            # Get the index of the max log-probability (prediction)
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count correct predictions
            correct += pred.eq(target.view_as(pred)).sum().item()

    # Print test results
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training the model
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302232

Test set: Average loss: 0.0004, Accuracy: 7224/10000 (72%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.785710

Test set: Average loss: 0.0003, Accuracy: 8351/10000 (84%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.655923

Test set: Average loss: 0.0003, Accuracy: 8954/10000 (90%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.581136

Test set: Average loss: 0.0003, Accuracy: 9108/10000 (91%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 1.556797

Test set: Average loss: 0.0003, Accuracy: 9198/10000 (92%)

